In [1]:
import openslide
from openslide import lowlevel as openslide_ll
from ctypes import c_uint32
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, Optional
from loguru import logger
import re
import cv2

from shipline.mpp import get_slide_mpp

SLIDE_FILE = "/data/data/TCGA-BRCA-DX-IMGS_1133/TCGA-AO-A12B-01Z-00-DX1.B215230B-5FF7-4B0A-9C1E-5F1658534B11.svs"

In [2]:
slide = openslide.OpenSlide(SLIDE_FILE)
target_mpp = 256./224.
slide_mpp = get_slide_mpp(slide)

2023-08-04 09:08:03.979 | INFO     | shipline.mpp:get_slide_mpp:42 - MPP successfully extracted using extract_mpp_from_properties: 0.499


In [5]:
def read_region(slide: openslide.OpenSlide, loc: Tuple[int, int], level: int, size: Tuple[int, int], resize: Optional[Tuple[int, int]] = None) -> np.ndarray:
    """Adapted from openslide.lowlevel.read_region() to not use PIL images, but directly return a numpy array."""
    x, y = loc
    w, h = size
    if w < 0 or h < 0:
        raise openslide.OpenSlideError(
            "negative width (%d) or negative height (%d) not allowed" % (w, h)
        )
    buf = (w * h * c_uint32)()
    openslide_ll._read_region(slide._osr, buf, x, y, level, w, h)
    openslide_ll._convert.argb2rgba(buf)
    img = np.frombuffer(buf, dtype=np.uint8).reshape(*size[::-1], 4)[..., :3] # remove alpha channel
    if resize is not None:
        img = cv2.resize(img, resize)
    return img

In [4]:
# Find biggest zoom level for given target MPP
level = int(np.log2(target_mpp / slide_mpp))
level_zoom_factor = 2**level

extraction_mpp = slide_mpp * level_zoom_factor

steps = 32
stride = np.ceil(np.array(slide.dimensions)/steps) # stride in pixels at level 0
stride = stride * level_zoom_factor # stride in pixels at extraction level
stride = stride.astype(int)
tile_target_size = stride*extraction_mpp/target_mpp # tile size in pixels at extraction level
tile_target_size = tile_target_size.astype(int)


In [6]:
%timeit -n 1 -r 1 read_region(slide, (0, 0), level, stride, tile_target_size)

1.01 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
